# Importing Libraries

In [1]:
# External imports
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw, ImageFont
from PIL.PngImagePlugin import PngImageFile
import requests
from textwrap import wrap

In [2]:
# Internal imports
from utils import import_xml_as_df

# 1. Importing XML as Pandas DataFrame

In [3]:
# Generate raw offers dataframe
df_offers_raw = import_xml_as_df(path='../resources/examples/LomadeeDownload_raw.xml')

In [4]:
# Display raw offers dataframe
display(df_offers_raw)

,offerName,sellerId,sellerThumbnail,offerLink,offerThumbnail,priceFrom,sellerName,priceTo,sku,categoryName,categoryId
0,Armário de Cozinha Compacta 180cm Branco Topaz...,7183,https://www.lomadee.com/programas/BR/7183/imag...,https://redir.lomadee.com/v2/27069d12b45,https://madesacms.vteximg.com.br/arquivos/ids/...,1189.9,Loja Madesa,921.4,216,Geral,0
1,Máquina de Lavar Electrolux 15kg Branca Essent...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/270cc7da44e,https://electrolux.vteximg.com.br/arquivos/ids...,3169.0,Electrolux,1999.0,2003125,Geral,0
2,Geladeira Electrolux Frost Free 382L Duplex Co...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/270f69ac2e6,https://electrolux.vteximg.com.br/arquivos/ids...,4689.0,Electrolux,2899.0,2002168,Geral,0
3,Geladeira Electrolux Frost Free 310L Duplex Br...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/270ebdec332,https://electrolux.vteximg.com.br/arquivos/ids...,3489.0,Electrolux,2499.0,2002169,Geral,0
4,Máquina de Lavar Electrolux 13kg Branca Essent...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/27067890937,https://electrolux.vteximg.com.br/arquivos/ids...,2839.0,Electrolux,1849.0,2003123,Geral,0
5,Lava e Seca Electrolux 11Kg/7Kg cor Inox Ultim...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/270ad9dd753,https://electrolux.vteximg.com.br/arquivos/ids...,6069.0,Electrolux,3999.0,2002565,Geral,0
6,Geladeira Electrolux Frost Free Inverter 431L ...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/270672e9f9d,https://electrolux.vteximg.com.br/arquivos/ids...,5079.0,Electrolux,3449.0,2002350,Geral,0
7,Notebook VAIO® FE15 AMD® Ryzen 7 Windows 11 Ho...,6126,https://www.lomadee.com/programas/BR/6126/imag...,https://redir.lomadee.com/v2/270a7f5f2e4,https://vaiobr.vteximg.com.br/arquivos/ids/161...,NaN,Vaio,5499.0,42569,Geral,0
8,"Smart Tv Lg 43"" Led 4k Thinq Ai Comando De Voz...",5727,https://www.lomadee.com/programas/BR/5727/imag...,https://redir.lomadee.com/v2/270342001d9,https://www.girafa.com.br/imagens/eletronico/t...,NaN,Girafa,1887.78,918842,Eletrônicos,1
9,Geladeira Electrolux Frost Free 382L Duplex Br...,6078,https://www.lomadee.com/programas/BR/6078/imag...,https://redir.lomadee.com/v2/27007703ed0,https://electrolux.vteximg.com.br/arquivos/ids...,4229.0,Electrolux,2749.0,2002102,Geral,0


# 2. Refining DataFrame

In [5]:
# Refine Dataframe (Social Soul) function
def refine_dataframe(df:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    # STEP 1: Select columns of interest
    # Set columns of interest
    columns_of_interest = [
        'offerName',
        #'sellerId',
        #'sellerThumbnail',
        'offerLink',
        'offerThumbnail',
        'priceFrom',
        #'sellerName',
        'priceTo',
        #'sku',
        #'categoryName',
        #'categoryId',
    ]
    # Select columns of interest only
    df = df.loc[:,columns_of_interest]
    
    
    # STEP 2: Cast prices from STR to FLOAT64
    df['priceFrom'] = df['priceFrom'].astype('float64')
    df['priceTo'] = df['priceTo'].astype('float64')
    
    
    # STEP 3: Clean absent values
    # Replace NaN (Not a Number) with None
    df = df.replace(np.nan, None)
    
    
    # Return refined dataframe
    return df

In [6]:
# Generate refined offers dataframe
df_offers_refined = refine_dataframe(df=df_offers_raw)

In [7]:
# Display refined offers dataframe
display(df_offers_refined)

,offerName,offerLink,offerThumbnail,priceFrom,priceTo
0,Armário de Cozinha Compacta 180cm Branco Topaz...,https://redir.lomadee.com/v2/27069d12b45,https://madesacms.vteximg.com.br/arquivos/ids/...,1189.9,921.40
1,Máquina de Lavar Electrolux 15kg Branca Essent...,https://redir.lomadee.com/v2/270cc7da44e,https://electrolux.vteximg.com.br/arquivos/ids...,3169.0,1999.00
2,Geladeira Electrolux Frost Free 382L Duplex Co...,https://redir.lomadee.com/v2/270f69ac2e6,https://electrolux.vteximg.com.br/arquivos/ids...,4689.0,2899.00
3,Geladeira Electrolux Frost Free 310L Duplex Br...,https://redir.lomadee.com/v2/270ebdec332,https://electrolux.vteximg.com.br/arquivos/ids...,3489.0,2499.00
4,Máquina de Lavar Electrolux 13kg Branca Essent...,https://redir.lomadee.com/v2/27067890937,https://electrolux.vteximg.com.br/arquivos/ids...,2839.0,1849.00
5,Lava e Seca Electrolux 11Kg/7Kg cor Inox Ultim...,https://redir.lomadee.com/v2/270ad9dd753,https://electrolux.vteximg.com.br/arquivos/ids...,6069.0,3999.00
6,Geladeira Electrolux Frost Free Inverter 431L ...,https://redir.lomadee.com/v2/270672e9f9d,https://electrolux.vteximg.com.br/arquivos/ids...,5079.0,3449.00
7,Notebook VAIO® FE15 AMD® Ryzen 7 Windows 11 Ho...,https://redir.lomadee.com/v2/270a7f5f2e4,https://vaiobr.vteximg.com.br/arquivos/ids/161...,None,5499.00
8,"Smart Tv Lg 43"" Led 4k Thinq Ai Comando De Voz...",https://redir.lomadee.com/v2/270342001d9,https://www.girafa.com.br/imagens/eletronico/t...,None,1887.78
9,Geladeira Electrolux Frost Free 382L Duplex Br...,https://redir.lomadee.com/v2/27007703ed0,https://electrolux.vteximg.com.br/arquivos/ids...,4229.0,2749.00


# 3. Creating Instagram Stories images

In [8]:
# Add Text To Image function
def add_text_to_image(
    im:PngImageFile,
    font:str = '../resources/fonts/OpenSans-VariableFont_wdth,wght.ttf',
    font_size:int = 12,
    x:int = 0,
    y:int = 0,
    text:str = 'No text provided',
    text_align:str = 'left',
):
    
    # Get a drawing context
    draw = ImageDraw.Draw(im=im)
    
    # Set font
    text_font = ImageFont.truetype(
        font=font,
        size=font_size,
        encoding='unic'
    )
    
    # Add text to image
    draw.multiline_text(
        xy=(x,y),
        text=text,
        fill=(0,0,0),
        font=text_font,
        align=text_align,
        stroke_width=0
    )
    
    # Return image (with added text)
    return im
    

In [9]:
def generate_ig_stories_image(
    offer_thumbnail:str = None,
    offer_name:str = None,
    offer_price_from:str = None,
    offer_price_to:str = None,
    file_path:str = None
) -> str:
    
    # STEP 1: Create empty IG Stories image from template
    # Create empty story from template
    im_ig_stories = Image.open(fp='../resources/story_template_720x1280_final.png')
    
    
    # STEP 2: Add offer thumbnail to IG Stories image
    # Get offer thumbnail
    im_thumbnail = Image.open(requests.get(offer_thumbnail, stream=True).raw)
    # Resize offer thumbnail
    im_thumbnail = im_thumbnail.resize(size=(640,640))
    # Paste offer thumbnail to story image
    im_ig_stories.paste(im=im_thumbnail,box=(40, 130))
    
    
    # STEP 3: Add offer name to IG Stories image
    # Add offer name
    im_ig_stories = add_text_to_image(
        im=im_ig_stories,
        font_size=40,
        x=27.5, # (10+17.5)
        y=827.5, # (100+700+20+7.5)
        text= '\n'.join(wrap(text=offer_name, width=30)),
        text_align='left'
    )
    
    
    # STEP 4: Add offer price to IG Stories image
    # If no priceFrom:
    if offer_price_from == None:
        price_text = f'Por apenas R${offer_price_to:.2f}!'
    # Else (if priceFrom):
    else:
        price_text = f'De R${offer_price_from:.2f} por apenas R${offer_price_to:.2f}!'
    # Add offer price to IG Stories image
    im_ig_stories = add_text_to_image(
        im=im_ig_stories,
        font_size=37.5,
        x=27.5, # (10+17.5)
        y=1062.5, # (100+700+20+210+20+12.5)
        text=price_text,
        text_align='left'
    )
    
    
    # STEP 5: Save IG Stories image to temp folder
    im_ig_stories.save(fp=file_path, format='png')
    
    
    # Return path fo IG Stories image
    return file_path
    

In [10]:
# Add Instagram Stories Images To Dataframe function
def add_ig_stories_images_to_df(df_offers:pd.core.frame.DataFrame) -> pd.core.frame.DataFrame:
    
    # Create empty list of IG Stories images
    ig_stories_images = []
    
    # For each offer in offers dataframe:
    for i in range(len(df_offers)):
        
        # Generate IG Stories image
        ig_stories_images.append(
            generate_ig_stories_image(
                offer_thumbnail= df_offers.loc[i, 'offerThumbnail'],
                offer_name= df_offers.loc[i, 'offerName'],
                offer_price_from= df_offers.loc[i, 'priceFrom'],
                offer_price_to= df_offers.loc[i, 'priceTo'],
                file_path= f'../temp/ig_stories_images/offer_{i}.png',
            )
        )
    
    # Add IG Stories images list as column in dataframe        
    df_offers['storiesImage'] = ig_stories_images
    
    # Return dataframe
    return df_offers

In [11]:
df_offers_ready = add_ig_stories_images_to_df(df_offers=df_offers_refined)

In [12]:
display(df_offers_refined)

,offerName,offerLink,offerThumbnail,priceFrom,priceTo,storiesImage
0,Armário de Cozinha Compacta 180cm Branco Topaz...,https://redir.lomadee.com/v2/27069d12b45,https://madesacms.vteximg.com.br/arquivos/ids/...,1189.9,921.40,../temp/ig_stories_images/offer_0.png
1,Máquina de Lavar Electrolux 15kg Branca Essent...,https://redir.lomadee.com/v2/270cc7da44e,https://electrolux.vteximg.com.br/arquivos/ids...,3169.0,1999.00,../temp/ig_stories_images/offer_1.png
2,Geladeira Electrolux Frost Free 382L Duplex Co...,https://redir.lomadee.com/v2/270f69ac2e6,https://electrolux.vteximg.com.br/arquivos/ids...,4689.0,2899.00,../temp/ig_stories_images/offer_2.png
3,Geladeira Electrolux Frost Free 310L Duplex Br...,https://redir.lomadee.com/v2/270ebdec332,https://electrolux.vteximg.com.br/arquivos/ids...,3489.0,2499.00,../temp/ig_stories_images/offer_3.png
4,Máquina de Lavar Electrolux 13kg Branca Essent...,https://redir.lomadee.com/v2/27067890937,https://electrolux.vteximg.com.br/arquivos/ids...,2839.0,1849.00,../temp/ig_stories_images/offer_4.png
5,Lava e Seca Electrolux 11Kg/7Kg cor Inox Ultim...,https://redir.lomadee.com/v2/270ad9dd753,https://electrolux.vteximg.com.br/arquivos/ids...,6069.0,3999.00,../temp/ig_stories_images/offer_5.png
6,Geladeira Electrolux Frost Free Inverter 431L ...,https://redir.lomadee.com/v2/270672e9f9d,https://electrolux.vteximg.com.br/arquivos/ids...,5079.0,3449.00,../temp/ig_stories_images/offer_6.png
7,Notebook VAIO® FE15 AMD® Ryzen 7 Windows 11 Ho...,https://redir.lomadee.com/v2/270a7f5f2e4,https://vaiobr.vteximg.com.br/arquivos/ids/161...,None,5499.00,../temp/ig_stories_images/offer_7.png
8,"Smart Tv Lg 43"" Led 4k Thinq Ai Comando De Voz...",https://redir.lomadee.com/v2/270342001d9,https://www.girafa.com.br/imagens/eletronico/t...,None,1887.78,../temp/ig_stories_images/offer_8.png
9,Geladeira Electrolux Frost Free 382L Duplex Br...,https://redir.lomadee.com/v2/27007703ed0,https://electrolux.vteximg.com.br/arquivos/ids...,4229.0,2749.00,../temp/ig_stories_images/offer_9.png
